 Zillow 2016 Regression Analysis

## Project Planning

### Goals

(see background)...your customer is the zillow data science team.  state your goals as if you were delivering this to zillow.  They have asked for something from you (see 'background') and you are basically communicating in a more concise way, and very clearly, the goals as you understand them and as you have taken and acted upon through your research. 

*Project Overview
Background:
Zillow wants to improve their Zestimate. The zestimate is estimated value of a home. Zillow theorizes that there is more information to be gained to improve its existing model. Because of that, Zillow wants you to develop a model to predict the error between the Zestimate and the sales price of a home. In predicting the error, you will discover features that will help them improve the Zestimate estimate itself. Your goal of this project is to develop a linear regression model that will best predict the log error of the Zestimate. The error is the difference of the sales price and the Zestimate. The log error is computed by taking the log function of that error. You don't need to worry about the fact that the error is of a logarithmic function. It is a continuous number that represents an error rate.*



### Deliverables

What should the zillow team expect to receive from you?  Again, as you were communicating to them, not to your instructors.  

*your deliverables:*
1. *A report (in the form of a presentation, both verbal and through a slides) that summarizes your findings about the drivers of the Zestimate error. This will come from the analysis you do during the exploration phase of the pipeline. In the report, you will have charts that visually tell the story of what is driving the errors.*

2. *A Jupyter notebook, titled 'Regression_Proj_YourName', that contains a clearly labeled section and code clearly documented for each the stages below (project planning, data acquisition, data prep, exploration, and modeling). All of the work will take place in your jupyter notebook.*

### Data Dictionary & Details

**we will be using 2016 data so do NOT use properties_2017 or predictions_2017**  

Data dictionary of fields you will use. Why? So that you can refer back and others can refer to the meanings as you are developing your model. This is about gaining knowledge in the domain space so that you will understand when data doesn't look right, be able to more effectively develop hypotheses, and use that domain knowledge to build a more robust model (among other reasons)

Define your fields.   You may be able to to some ahead of time, but you may need to return to this section after you have explored the data and understand more about what each field means.  Also, look for a data dictionary either in the database or in the original data source (kaggle.com). 

You are free to use more fields than is mentioned, but I would recommend not expanding it to too many fields in the beginning as it will add complexity and you want to make sure you get an initial version completed before diving in deeper.  

- logerror
- bathroomcnt
- bedroomcnt
- calculatedfinishedsquarefeet
- fullbathcnt
- garagecarcnt
- roomcnt
- yearbuilt
- taxvaluedollarcnt
- taxamount

### Initial Thoughts, Ideas, Hypotheses

Brainstorming ideas, hypotheses, related to how variables might impact or relate to each other, both within independent variables and between the independent variables and dependent variable, and also related to any ideas for new features you may have while first looking at the existing variables and challenge ahead of you.

### Prepare your environment

import libraries you will use throughout the project.  You may need to add to this as you go.  But try to keep all of your imports in this section

In [16]:
import pandas as pd
import scipy.stats as stats
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('classic')
import statsmodels.api as sm
from scipy.stats import pearsonr
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, median_absolute_error
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import SelectKBest, f_classif
from env import user, host, pw, datadirpath

### Data Acquisition

#### Acquire via csv

- add the path to where your files are located in your env.py file. 
- ensure that others can read the files based on their local env.py file. 

- Filter out any from predictions_2016 that do not have a logerror. 
- Generate a cohesive data set that includes the following fields:

- `logerror`
- `bathroomcnt`
- `bedroomcnt`
- `calculatedfinishedsquarefeet`
- `fullbathcnt`
- `garagecarcnt`
- `roomcnt`
- `yearbuilt`
- `taxvaluedollarcnt`
- `taxamount`
- (optional) `regionidzip`

recommendations for dealing with the large dataset
- remove any without a logerror
- sample until you have the right query (and then pull entire dataset)
- export to local csv 
- read from local csv (sample, but larger sample than your first) 
- save sql query to add into to_sql() function in python. 


#### Summarize the data. 

Why? So you can confirm the data look like what you would expect.

- peek at a few rows
- data types
- summary stats
- column names
- number of rows and columns
- ...

Make a re-useable list of columns to analyze, call it Features and feed to to pd.read.csv to create the dataframe only containing the needed colums.   Note:  If you decide on different features - change the "FeatureList" and rerun

In [17]:
FeatureList = ['bathroomcnt','bedroomcnt','calculatedfinishedsquarefeet','fullbathcnt','garagecarcnt','roomcnt','yearbuilt','taxvaluedollarcnt','taxamount','regionidzip','lotsizesquarefeet']
columnstopull = ['parcelid'] + FeatureList
csvfilename = "properties_2016.csv"
properties = pd.read_csv(datadirpath+csvfilename, usecols=columnstopull)
properties.head()


,parcelid,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fullbathcnt,garagecarcnt,lotsizesquarefeet,regionidzip,roomcnt,yearbuilt,taxvaluedollarcnt,taxamount
0,10754147,0.0,0.0,NaN,NaN,NaN,85768.0,96337.0,0.0,NaN,9.0,NaN
1,10759547,0.0,0.0,NaN,NaN,NaN,4083.0,96337.0,0.0,NaN,27516.0,NaN
2,10843547,0.0,0.0,73026.0,NaN,NaN,63085.0,96095.0,0.0,NaN,1413387.0,20800.37
3,10859147,0.0,0.0,5068.0,NaN,NaN,7521.0,96424.0,0.0,1948.0,1156834.0,14557.57
4,10879947,0.0,0.0,1776.0,NaN,NaN,8512.0,96450.0,0.0,1947.0,433491.0,5725.17


In [18]:
csvfilename = "train_2016_v2.csv"
logerrors = pd.read_csv(datadirpath+csvfilename)
logerrors.head()

,parcelid,logerror,transactiondate
0,11016594,0.0276,2016-01-01
1,14366692,-0.1684,2016-01-01
2,12098116,-0.0040,2016-01-01
3,12643413,0.0218,2016-01-02
4,14432541,-0.0050,2016-01-02


    Using pd.merge on the dataframes and checking for duplicates validate="one_to_one",   there are duplicates
use pd.concat to find out how many duplicated parcelids.    There are 249 duplicated parcelid s
    Inspection of the duplicates indicates that these are corrected rows, with a later transaction date always occuring on the last duplicate.   

In [23]:
pd.concat(dupes for _, dupes in logerrors.groupby("parcelid") if len(dupes) > 1)

,parcelid,logerror,transactiondate
10828,10736972,-0.0460,2016-02-22
10829,10736972,-0.0367,2016-12-14
9177,10790468,-0.0450,2016-02-12
9178,10790468,-0.0440,2016-09-13
2226,10796614,-0.1267,2016-01-13
2227,10796614,-0.0704,2016-06-14
24522,10798910,-0.0471,2016-04-12
24523,10798910,-0.0274,2016-05-24
6291,10799924,0.0478,2016-01-29
6292,10799924,0.4121,2016-08-09


Now remove the duplicates (keeping the last occurance) based on parcel id

In [25]:
logerrors.drop_duplicates(subset='parcelid', keep='last', inplace=True)

90150

  and merge the two dataframes, checking for duplicates 

In [49]:
frames = [properties, logerrors]
df = pd.merge(properties, logerrors, on='parcelid', how='inner', validate="one_to_one")
df.head()

,parcelid,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fullbathcnt,garagecarcnt,lotsizesquarefeet,regionidzip,roomcnt,yearbuilt,taxvaluedollarcnt,taxamount,logerror,transactiondate
0,17073783,2.5,3.0,1264.0,2.0,2.0,1735.0,97081.0,5.0,1986.0,191811.0,2015.06,0.0953,2016-01-27
1,17088994,1.0,2.0,777.0,1.0,1.0,NaN,97083.0,4.0,1990.0,239679.0,2581.30,0.0198,2016-03-30
2,17100444,2.0,3.0,1101.0,2.0,2.0,6569.0,97113.0,5.0,1956.0,47853.0,591.64,0.0060,2016-05-27
3,17102429,1.5,2.0,1554.0,1.0,2.0,7400.0,97113.0,5.0,1965.0,62914.0,682.78,-0.0566,2016-06-07
4,17109604,2.5,4.0,2415.0,2.0,2.0,6326.0,97084.0,8.0,1984.0,554000.0,5886.92,0.0573,2016-08-08


### Data Preparation

Goal: leave this section with a dataset that is ready to be analyzed. 

#### Create a variable, `colnames`, that is a list of the column names. 
Why? You will likely reference this variable later.

In [37]:
FeatureList

['bathroomcnt',
 'bedroomcnt',
 'calculatedfinishedsquarefeet',
 'fullbathcnt',
 'garagecarcnt',
 'roomcnt',
 'yearbuilt',
 'taxvaluedollarcnt',
 'taxamount',
 'regionidzip',
 'lotsizesquarefeet']

#### Identify the data types of each variable. Why? 
You need to make sure they are what makes sense for the data and the meaning of the data that lies in that variable. If it does not, make necessary changes.

In [50]:
df.describe()

,parcelid,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fullbathcnt,garagecarcnt,lotsizesquarefeet,regionidzip,roomcnt,yearbuilt,taxvaluedollarcnt,taxamount,logerror
count,9.015000e+04,90150.000000,90150.000000,89492.000000,88974.000000,29897.000000,8.001400e+04,90115.000000,90150.000000,89397.000000,9.014900e+04,90144.000000,90150.000000
mean,1.298488e+07,2.279545,3.031936,1773.096869,2.241172,1.812055,2.912064e+04,96586.196216,1.478580,1968.539761,4.576379e+05,5983.070888,0.011526
std,2.505517e+06,1.004133,1.156114,928.136339,0.963106,0.608865,1.217909e+05,3663.847246,2.819802,23.763165,5.548532e+05,6838.506814,0.160968
min,1.071174e+07,0.000000,0.000000,2.000000,1.000000,0.000000,1.670000e+02,95982.000000,0.000000,1885.000000,2.200000e+01,49.080000,-4.605000
25%,1.155970e+07,2.000000,2.000000,1184.000000,2.000000,2.000000,5.704000e+03,96193.000000,0.000000,1953.000000,1.990560e+05,2873.260000,-0.025300
50%,1.254744e+07,2.000000,3.000000,1540.000000,2.000000,2.000000,7.200000e+03,96393.000000,0.000000,1970.000000,3.429310e+05,4543.100000,0.006000
75%,1.422757e+07,3.000000,4.000000,2095.000000,3.000000,2.000000,1.168175e+04,96987.000000,0.000000,1987.000000,5.405890e+05,6900.165000,0.039200
max,1.629608e+08,20.000000,16.000000,22741.000000,20.000000,24.000000,6.971010e+06,399675.000000,18.000000,2015.000000,2.775000e+07,321936.090000,4.737000


#### Identify the columns that have missing values and the number of missing values in each column. 
Why? Missing values are going to cause issues down the line so you will need to handle those appropriately. For each variable with missing values, if it makes sense to replace those missing with a 0, do so. For those where that doesn't make sense, decide if you should drop the entire observations (rows) that contain the missing values, or drop the entire variable (column) that contains the missing values.

In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 90150 entries, 0 to 90149
Data columns (total 14 columns):
parcelid                        90150 non-null int64
bathroomcnt                     90150 non-null float64
bedroomcnt                      90150 non-null float64
calculatedfinishedsquarefeet    89492 non-null float64
fullbathcnt                     88974 non-null float64
garagecarcnt                    29897 non-null float64
lotsizesquarefeet               80014 non-null float64
regionidzip                     90115 non-null float64
roomcnt                         90150 non-null float64
yearbuilt                       89397 non-null float64
taxvaluedollarcnt               90149 non-null float64
taxamount                       90144 non-null float64
logerror                        90150 non-null float64
transactiondate                 90150 non-null object
dtypes: float64(12), int64(1), object(1)
memory usage: 10.3+ MB


Cleanup,   1) drop column "garagecarcnt' completelty because only one-third of the rows have values 
           2) drop rows containing null values 
           3) change parcelid, regionidzip, yearbuilt to strings.     
           4) change lotsizesquarefeet to acres to make it human readable (43650 sq feet per acre)
           5) find a reasonable threshold to drop rows where calculatedfinishedsquarefeet has a minimum value
               aka - 2 square feet is not reasonable
           6) check for roomcnt, values don't look correct (min, IQR nonsensical)           

In [52]:
df.drop(columns=['garagecarcnt'], inplace=True)

In [53]:
df.dropna(how='any',axis=0, inplace=True)

In [60]:
df['parcelid'] = df['parcelid'].apply(lambda x: "{:.0f}".format(x) if x is not None else x)

In [55]:
df['regionidzip'] = df['regionidzip'].apply(lambda x: "{:.0f}".format(x) if x is not None else x)

In [57]:
df['yearbuilt'] = df['yearbuilt'].apply(lambda x: "{:.0f}".format(x) if x is not None else x)

In [62]:
df.lotsizesquarefeet = df.lotsizesquarefeet / 43650

In [65]:
df.rename(columns={"lotsizesquarefeet":"lotsizeacres"}, inplace=True)

In [66]:
df.head()

,parcelid,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fullbathcnt,lotsizeacres,regionidzip,roomcnt,yearbuilt,taxvaluedollarcnt,taxamount,logerror,transactiondate
0,17073783,2.5,3.0,1264.0,2.0,0.039748,97081,5.0,1986,191811.0,2015.06,0.0953,2016-01-27
2,17100444,2.0,3.0,1101.0,2.0,0.150493,97113,5.0,1956,47853.0,591.64,0.0060,2016-05-27
3,17102429,1.5,2.0,1554.0,1.0,0.169530,97113,5.0,1965,62914.0,682.78,-0.0566,2016-06-07
4,17109604,2.5,4.0,2415.0,2.0,0.144926,97084,8.0,1984,554000.0,5886.92,0.0573,2016-08-08
5,17125829,2.5,4.0,2882.0,2.0,0.229095,97089,8.0,1980,289609.0,3110.44,0.0564,2016-08-26


In [68]:
df.describe()

,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fullbathcnt,lotsizeacres,roomcnt,taxvaluedollarcnt,taxamount,logerror
count,79079.000000,79079.000000,79079.000000,79079.000000,79079.000000,79079.000000,7.907900e+04,79079.000000,79079.000000
mean,2.325232,3.156995,1817.884331,2.272234,0.660922,1.504167,4.651417e+05,6090.813175,0.012012
std,1.001983,1.108746,948.850148,0.985885,2.712144,2.883554,5.778778e+05,7091.143985,0.160558
min,1.000000,0.000000,2.000000,1.000000,0.003826,0.000000,7.704000e+03,49.080000,-4.605000
25%,2.000000,2.000000,1210.000000,2.000000,0.130882,0.000000,1.942010e+05,2884.000000,-0.026300
50%,2.000000,3.000000,1577.000000,2.000000,0.164948,0.000000,3.428710e+05,4594.950000,0.006000
75%,3.000000,4.000000,2163.000000,3.000000,0.268041,0.000000,5.530120e+05,6997.325000,0.039200
max,20.000000,16.000000,22741.000000,20.000000,159.702405,18.000000,2.775000e+07,321936.090000,4.737000


In [80]:
# smallhouses = df.calculatedfinishedsquarefeet.(calculatedfinishedsquarefeet < 20)

smallhouses = df[df['calculatedfinishedsquarefeet'] < 200]
smallhouses

,parcelid,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fullbathcnt,lotsizeacres,regionidzip,roomcnt,yearbuilt,taxvaluedollarcnt,taxamount,logerror,transactiondate
14927,10902671,3.0,4.0,120.0,3.0,0.341833,96452,0.0,1939,788975.0,9622.16,0.0602,2016-04-26
15107,12561798,1.0,0.0,199.0,1.0,0.767400,96236,0.0,1928,102553.0,1358.83,0.1017,2016-09-19
49170,10825953,20.0,3.0,66.0,20.0,0.154616,96414,0.0,1960,233496.0,2831.76,-0.0325,2016-08-31
81326,12660907,3.0,0.0,2.0,3.0,0.229118,96123,0.0,1980,337871.0,4163.62,0.0440,2016-03-25
84905,12236415,2.0,3.0,152.0,2.0,0.124284,96025,0.0,1939,85122.0,1565.27,0.0050,2016-05-26


#### Create a list of the independent variable names (aka attributes) and assign it to `x_vars` 
Why? During exploration, you will likely use this list to refer to the attribute names.

#### Clearly identify your dependent (target) variable. 
- What is the name of the variable? 
- Is it discrete or continuous?

#### View the distribution of the numeric variables
- plot a histogram and box plot of each variable. 
- Why? To see the distribution, skewness, outliers, and unit scales. You will use this information in your decision of whether to normalize, standardize or neither.

#### BONUS:  Data standardization 
Create a new data frame that is the min-max normalization of the independent variable in the original data frame (+ the original dependent variable). You will normalize each of the numeric independent variables independently, i.e. using the min and max of each variable, not the min/max of the whole dataframe. Why? Regression is very sensitive to difference in units. It will be almost impossible to extract a meaningful linear regression model with such extreme differences in scale. For more context, see: https://medium.com/@rrfd/standardize-or-normalize-examples-in-python-e3f174b65dfc
You will use this dataframe in the future analysis.

### Data Exploration

Goal is to address each of the questions you posed in your planning & brainstorming through visual or statistical analysis.

When you have completed this step, you will have the findings from your analysis that will be used in your final report, the answers to your questions and your customer's questions that will reach the goal. 

#### Split data into training and test datasets

#### Create a jointplot for each independent variable (normalized version) with the dependent variable. 

Be sure you have Pearson's r and p-value annotated on each plot.

#### Create a feature plot using seaborn's PairGrid() of the interaction between each variable (dependent + independent). 

You may want to use a normalized dataframe (if you did that) or adjust the axis scale (set to logarithmic, e.g) so you can more clearly view the interactions.

#### Create a heatmap of the correlation between each variable pair.

#### Summarize your conclusions from all of these steps.  

#### Is the logerror significantly different for homes with 3 bedrooms vs those with 5 or more bedrooms? 
Run a t-test to test this difference.

#### Do the same for another 2 samples you are interested in comparing (e.g. those with 1 bath vs. x baths)

### Data Modeling

#### Feature Engineering & Selection

#### Are there new features you could create based on existing features that might be helpful? 

Come up with at least one possible new feature that is a calculation from 2+ existing variables. 

#### Use statsmodels ordinary least squares to assess the importance of each feature with respect to the target

#### Summarize your conclusions and next steps from your analysis in above. 
What will you try when developing your model? (which features to use/not use/etc)

#### Train & Test Model

#### Fit, predict (in-sample) & evaluate multiple linear regression models to find the best one.

- Make any changes as necessary to improve your model.

- Identify the best model after all training

- Compare the models by plotting the predictions of each model with actual values (see plot right above section 12 in 'Regression in Python' lesson

(the predictions are a 1 x 83 two dimensional matrix, but we want just a single array of predictions. We can use the .ravel method to achieve this.)

`pd.DataFrame({'actual': y_train.final_grade,  
              'lm1': y_pred_lm1.ravel(),  
              'lm2': y_pred_lm2.ravel()})\  
    .melt(id_vars=['actual'], var_name='model', value_name='prediction')\  
    .pipe((sns.relplot, 'data'), x='actual', y='prediction', hue='model')  

plt.plot([60, 100], [60, 100], c='black', ls=':')  
plt.ylim(60, 100)  
plt.xlim(60, 100)  
plt.title('Predicted vs Actual Final Grade')`  

#### Predict & evaluate on out-of-sample data (test)    

- Plot the residuals from your out-of-sample predictions.

- Summarize your expectations about how you estimate this model will perform in production.